In [1]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from GoogleScraper import scrape_with_config, GoogleSearchError
from GoogleScraper.database import ScraperSearch, SERP, Link
import requests
import time

# 1. Configure Google Search

In [2]:
config = {
    'keyword': 'what should i site:answers.yahoo.com',
    'num_pages_for_keyword': 100,
    'scrape_method': 'http',
    'do_caching': True,
    'output_filename': 'Results/google.json',
    'google_search_params': {'cr': 'countryUS'}
}

# 2. Get Google Results

In [ ]:
try:
    search = scrape_with_config(config)
except GoogleSearchError as e:
    print(e)

# 3. Extract Links

In [4]:
links = []
for serp in search.serps:
    for link in serp.links:
        #if link.link.startswith('https://answers.yahoo.com'):
            links.append(link.link.strip())

# 4. Scrape Links

In [5]:
scrape = []
for link in links:
    response = requests.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    question_f = soup.find("h1", { "itemprop" : "name" })
    follows_f = soup.find("span", {"class","follow-text"})
    answers_f = soup.find("span", { "itemprop" : "answerCount" })
    topic_f = soup.find("div", {"itemprop" : "breadcrumb"})
    
    if question_f is not None:
        question = question_f.text
        topic = topic_f.text
        follows = -1
        answers = -1
        if follows_f is not None:
            follows = follows_f["data-ya-fc"]
        if answers_f is not None:
            answers = answers_f.text
        
        scrape.append([link, question.strip(), follows, answers, topic.strip()])
    else:
        scrape.append([link, None, None, None, None])
        

# 5. Output Results as CSV

In [6]:
scrapeDF = pd.DataFrame(scrape, columns=['Link', 'Question', 'Follows', 'Answers', 'Topic'])
scrapeDF.to_csv('Results/Yahoo! Answers Scrape Results_What Should I w Topic' + time.strftime("%Y%m%d-%H%M%S") + '.csv', index=False)

# END